<a href="https://colab.research.google.com/github/jamespatrickmanning/compare_models/blob/master/get_model_temp_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title zlconversions

# -*- coding: utf-8 -*-
"""
Created on Fri Sep 14 15:08:40 2018
update in MARCH 6 add a function find_nd(find the index of nearest distance)
@author: leizhao

directory list in the end

"""
#from __future__ import unicode_literals
#import platform
#import warnings

import re
import pandas as pd
import pytz
import datetime
import os,shutil
import numpy as np
import math
import difflib
import time
import requests

def angle_conversion(a):
    a = np.array(a)
    return a/180*np.pi
    
def copyfile(srcfile,dstfile):
    """copy file from one folder to another folder"""
    if not os.path.isfile(srcfile):
        print ("%s not exist!"%(srcfile))
    else:
        fpath,fname=os.path.split(dstfile) 
        if not os.path.exists(fpath):
            os.makedirs(fpath)
        shutil.copyfile(srcfile,dstfile)    
   
def dist(lat1=0,lon1=0,lat2=0,lon2=0):
    """caculate the distance of two points, return miles
    the format of lat and  lon is 00.00 (dd not dm)"""
    conversion_factor = 0.62137119
    R = 6371.004
    lon1, lat1 = angle_conversion(lon1), angle_conversion(lat1)
    lon2, lat2 = angle_conversion(lon2), angle_conversion(lat2)
    l = R*np.arccos(np.cos(lat1)*np.cos(lat2)*np.cos(lon1-lon2)+\
                        np.sin(lat1)*np.sin(lat2))*conversion_factor
    return l

def ThreeD_dist(lat1=0,lon1=0,lat2=0,lon2=0,h1=0,h2=0):
    """caculate the distance of two points, return meters
    the lat lon format is dd, the unit of h is m"""
    R = 6371.004
    lon1, lat1 = angle_conversion(lon1), angle_conversion(lat1)
    lon2, lat2 = angle_conversion(lon2), angle_conversion(lat2)
    l = R*np.arccos(np.cos(lat1)*np.cos(lat2)*np.cos(lon1-lon2)+\
                        np.sin(lat1)*np.sin(lat2))
    distance=math.sqrt((1000*l)**2+(h1-h2)**2)
    return distance

def find_header_rows(path_name):
    """the lens of header"""
    original_file=pd.read_csv(path_name,nrows=12,names=['0','1','2','3','4','5'])
    for i in range(len(original_file['0'])):
        if original_file['0'][i]=='HEADING':
            header_rows=i
            break 
    return header_rows

def find_nd(target,lat,lon,lats,lons):
    
    """ Bisection method:find the index of nearest distance"""
    row=0
    maxrow=len(lats)-1
    col=len(lats[0])-1
    while col>=0 and row<=maxrow:
        distance=dist(lat1=lats[row][col],lat2=lat,lon1=lons[row][col],lon2=lon)
        if distance<=target:
            break
        elif abs(lats[row][col]-lat)<abs(lons[row][col]-lon):
            col-=1
        else:
            row+=1
    distance=dist(lat1=lats[row][col],lat2=lat,lon1=lons[row][col],lon2=lon)
    row_md,col_md=row,col  #row_md the row of minimum distance
    #avoid row,col out of range in next step
    if row<3:
        row=3
    if col<3:
        col=3
    if row>maxrow-3:
        row=maxrow-3
    if col>len(lats[0])-4:
        col=len(lats[0])-4
    for i in range(row-3,row+3,1):
        for j in range(col-3,col+3,1):
            distance_c=dist(lat1=lats[i][j],lat2=lat,lon1=lons[i][j],lon2=lon)
            if distance_c<=distance:
                distance=distance_c
                row_md,col_md=i,j
    return row_md,col_md



def fitting(point,lat,lon):
    """
    point represent many data include lat lon and z
    format:[[lat,lon,z],[lat1,lon1,z]...]
    """
#represent the value of matrix
    ISum = 0.0
    X1Sum = 0.0
    X2Sum = 0.0
    X1_2Sum = 0.0
    X1X2Sum = 0.0
    X2_2Sum = 0.0
    YSum = 0.0
    X1YSum = 0.0
    X2YSum = 0.0

    for i in range(0,len(point)):
        
        x1i=point[i][0]
        x2i=point[i][1]
        yi=point[i][2]

        ISum = ISum+1
        X1Sum = X1Sum+x1i
        X2Sum = X2Sum+x2i
        X1_2Sum = X1_2Sum+x1i**2
        X1X2Sum = X1X2Sum+x1i*x2i
        X2_2Sum = X2_2Sum+x2i**2
        YSum = YSum+yi
        X1YSum = X1YSum+x1i*yi
        X2YSum = X2YSum+x2i*yi

#  matrix operations
# _mat1 is the mat1 inverse matrix
    m1=[[ISum,X1Sum,X2Sum],[X1Sum,X1_2Sum,X1X2Sum],[X2Sum,X1X2Sum,X2_2Sum]]
    mat1 = np.matrix(m1)
    m2=[[YSum],[X1YSum],[X2YSum]]
    mat2 = np.matrix(m2)
    _mat1 =mat1.getI()
    mat3 = _mat1*mat2

# use list to get the matrix data
    m3=mat3.tolist()
    a0 = m3[0][0]
    a1 = m3[1][0]
    a2 = m3[2][0]
    y = a0+a1*lat+a2*lon

    return y



def fuzzyfinder(user_input, collection):
    suggestions = []
    pattern = '.*?'.join(user_input)    # Converts 'djm' to 'd.*?j.*?m'
    regex = re.compile(pattern)         # Compiles a regex.
    for item in collection:
        match = regex.search(item)      # Checks if the current item matches the regex.
        if match:
            suggestions.append((len(match.group()), match.start(), item))
    return [x for _, _, x in sorted(suggestions)] 

def get_doppio_url(date):
    url='http://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/his/runs/History_RUN_2018-11-12T00:00:00Z'
    return url.replace('2018-11-12',date)


def gmt_to_eastern(times_gmt):
    """GMT time converted to US Eastern Time"""
    eastern = pytz.timezone('US/Eastern')
    gmt = pytz.timezone('Etc/GMT')
    date = datetime.datetime.strptime(str(times_gmt),'%Y-%m-%d %H:%M:%S')
    date_gmt=gmt.localize(date)
    easterndate=date_gmt.astimezone(eastern)
    return easterndate


def  isConnected(address="http://server.arcgisonline.com/ArcGIS"):
    
    "check the internet"
    try:
        html = requests.get(address,timeout=2)
    except:
        return False
    return True

def keep_number(value,integer_num,decimal_digits):
    """keep the lens of value"""    
    #ouput data type is str
    data=str(value)
    if len(data.split('.'))==2:
        integer=data.split('.')[0]
        decimal=data.split('.')[1]
    else:
        integer=data
        decimal=[]
    if integer_num==all:
        integer=integer
    elif len(integer)>integer_num:
        integer=integer[len(integer)-integer_num:]
    elif len(integer)<integer_num:
        for i in range(integer_num-len(integer)):
            integer='0'+integer[:]
    if decimal_digits==all:
        decimal=decimal
    elif len(decimal)>decimal_digits:
        decimal=decimal[:decimal_digits]
    elif len(decimal)<decimal_digits:
        if decimal==[]:
            decimal='0'
            for i in range(decimal_digits-len(decimal)):
                decimal=decimal[:]+'0'
        else:
            for i in range(decimal_digits-len(decimal)):
                decimal=decimal[:]+'0'
    return str(integer+'.'+decimal)
       

def list_all_files(rootdir):
    """get all files' path and name in rootdirectory"""
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
           path = os.path.join(rootdir,list[i])
           if os.path.isdir(path):
              _files.extend(list_all_files(path))
           if os.path.isfile(path):
              _files.append(path)
    return _files


def list_sd2uv(s,d):
    """aim at the list transform the speed and direction data to the x,y components of the arrow vectors(u,v)"""
    u,v=np.zeros(len(s)),np.zeros(len(s))
    for i in range(len(s)):
        u[i],v[i]=sd2uv(s[i],d[i])
    return u,v
        
    
    
def list_uv2sd(u,v):
    """aim at the list transform the x,y components of the arrow vectors(u,v) to the speed and direction data"""
    s,d=np.zeros(len(u)),np.zeros(len(u))
    for i in range(len(u)):
        s[i],d[i]=uv2sd(u[i],v[i])
    return s,d

def local2utc(local_st):
    """
    the format of time is datetime: eg.:datetime.datetime(2019, 3, 7, 15, 50, 50)
    local time to utc time"""
    time_struct = time.mktime(local_st.timetuple())
    utc_st = datetime.datetime.utcfromtimestamp(time_struct)
    return utc_st


def nrows_len_to(fle,long,name,**kwargs):
    df=pd.read_csv(fle,names=['key','value1','value2','value3','value4','value5'])
    for i in range(len(df)):
        if len(df.iloc[i].dropna())!=long:
            break
    df=df[:i].dropna(axis=1)
    df.columns=name
    return df


def nrows_to(fle,line,name,**kwargs):
    """only read the header"""
    df=pd.read_csv(fle,names=['1','2','3','4','5','6'])
    for i in range(len(df['1'])):
        if df['1'][i]==line:
            break
    df=df[:i].dropna(axis=1)
    df.columns=name
    return df

    
def sd_list_mean(speeds,directions):
    """aim at the list about average of speed and direction"""
    u_total,v_total=0,0
    for a in range(len(speeds)):
        u,v=sd2uv(speeds[a],directions[a])
        u_total=u_total+u
        v_total=v_total+v
    u_mean=u_total/len(speeds)
    v_mean=v_total/len(speeds)
    WS,WD=uv2sd(u_mean,v_mean)
    return WS,WD


     
def sd2uv(s,d):
    """transform the speed and direction data to the x,y components of the arrow vectors(u,v)""" 
    u_t=math.sin(math.radians(d))
    v_t=math.cos(math.radians(d))
    if abs(u_t)==1:
        v=0
        u=float(s)*u_t
    elif abs(v_t)==1:
        u=0
        v=float(s)*v_t
    else:
        u=float(s)*u_t
        v=float(s)*v_t
    return u,v

def skip_len_to(fle,long,**kwargs):
    df=pd.read_csv(fle,names=['key','value1','value2','value3','value4','value5'])
    for i in range(len(df)):
        if len(df.iloc[i].dropna())!=long:
            break
    return pd.read_csv(fle,skiprows=i)


def skip_to(fle, line,**kwargs):
    """only read the data,not read the header"""
    if os.stat(fle).st_size <= 5:
        raise ValueError("File is empty")
    with open(fle) as f:
        pos = 0
        cur_line = f.readline()
        while not cur_line.startswith(line):
            pos = f.tell()
            cur_line = f.readline()
        f.seek(pos)
        return pd.read_csv(f, **kwargs)


def str_similarity_ratio(str1,str2):
    """caculate the rato of similarity in two string """
    return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

def transform_date(date):
    """format the time to 10/26/2018"""
    date=date.replace(' ','')
    if len(date.split('/'))!=3:
        date=date.split('/')[0]+'/'+'01'+'/'+date.split('/')[1]
    if len(date.split('/')[0])==1:
        date='0'+date.split('/')[0]+'/'+date.split('/')[1]+'/'+date.split('/')[2]
    if len(date.split('/')[1])==1:
        date=date.split('/')[0]+'/'+'0'+date.split('/')[1]+'/'+date.split('/')[2]
    if len(date.split('/')[2])==2:
        date=date.split('/')[0]+'/'+date.split('/')[1]+'/'+'20'+date.split('/')[2]
    date_data=date.split('/')[2]+date.split('/')[0]+date.split('/')[1]
    return date_data

def utc2local(utc_st):
    """
    utc_st: the format like this: datetime.datetime(2019, 3, 7, 10, 50, 50)
    UTC time to local time
    """
    now_stamp = time.time()
    local_time = datetime.datetime.fromtimestamp(now_stamp)
    utc_time = datetime.datetime.utcfromtimestamp(now_stamp)
    offset = local_time - utc_time
    local_st = utc_st + offset
    return local_st

def uv2sd(u,v):
    """transform the x,y components of the arrow vectors(u,v) to the speed and direction data"""
#    s=math.sqrt(u**2+v**2)
    s=math.sqrt(np.square(u)+np.square(v))
    if s==0:
        d=0
    else:
        if abs(v/s)==1:
            d=180/np.pi*math.acos(float(v/s))
        elif abs(u/s)==1:
            d=180/np.pi*math.asin(float(u/s))
        else:
            dt=180/np.pi*math.atan(float(u/v))
            if u>0 and v>0:
                d=dt
            elif v<0:
                d=180+dt
            else:
                d=360+dt
    return s,d

  

In [0]:
#@title gomofs
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 25 15:37:42 2019
gets the output from Gulf of Maine Ocean Forecast System with function get_gomofs
@author: Lei Zhao with some help from Vitalli and JiM
Requires his "zlconversions" module in this same directory

Modification: March 15, 2019 - added a function(get_gomofs_url_forcast(date,forecastdate=1))
Modification: Feb 28, 2020 - made a much simpler function "get_gomofs" to get bottom temp and renamed Lei Zhao's as "get_gomofs_zl" 
Note: We might come back to Lei Zhao fancier way to fit the data but here we use the nearest node.
"""
import netCDF4
import datetime
#import zlconversions as zl
import numpy as np
import math
import time


def get_gomofs_url(date):
    """
    input: "date" as datetime.datetime(2019, 2, 27, 11, 56, 51)
    output: a url for gomofs output
    """
    hours=date.hour
    date_str=date.strftime('%Y%m%d%H%M%S')
    tn=int(math.floor((hours)/6.0)*6)
    tstr='t'+str(tn).zfill(2)+'z'  ## for example: t12z the number is 12
    #if round((hours)/3.0-1.5,0)==tn/3:
    if tn>=6: 
        nstr='n006'       # nstr in url represent nowcast string: n003 or n006
    else:
        nstr='n003'
    url='http://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/GOMOFS/MODELS/'\
    +date_str[:6]+'/nos.gomofs.fields.'+nstr+'.'+date_str[:8]+'.'+tstr+'.nc'
    return url

def get_gomofs_url_forecast(date,forecastdate=True):
    """
    same as get_gomofs_url but gets the forecast file instead of the nowcast
    where "date" is a datatime like datetime.datetime(2019, 2, 27, 11, 56, 51, 666857)
    forecastdate like date or True
    return the url of data
    """
    if forecastdate==True:  #if forcastdate is True: default the forcast date equal to the time of choose file.
        forecastdate=date
    date=date-datetime.timedelta(hours=1.5)  #the parameter of calculate txx(eg:t00,t06 and so on)
    tn=int(math.floor(date.hour/6.0)*6)  #the numer of hours in time index: eg: t12, the number is 12
    ymdh=date.strftime('%Y%m%d%H%M%S')[:10]  #for example:2019011112(YYYYmmddHH)
    tstr='t'+str(tn).zfill(2)+'z'  #tstr: for example: t12
    fstr='f'+str(3+3*math.floor((forecastdate-datetime.timedelta(hours=1.5+tn)-datetime.datetime.strptime(ymdh[:8],'%Y%m%d')).seconds/3600./3.)).zfill(3)#fnstr:the number in forcast index, for example f006 the number is 6
    url='http://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/GOMOFS/MODELS/'\
    +ymdh[:6]+'/nos.gomofs.fields.'+fstr+'.'+ymdh[:8]+'.'+tstr+'.nc'
    return url
def get_gomofs(date_time,lat,lon,mindistance=20):# JiM's simple version for bottom temp
    """
    JiM's simplified version of Lei Zhao's function gets only bottom temp
    the format time(GMT) is: datetime.datetime(2019, 2, 27, 11, 56, 51, 666857)
    lat and lon use decimal degrees where lon is negative number
    returns the BOTTOM temperature (degC) of specify location
    HARDCODED TO RETURN BOTTOM TEMP
    """
    rho_index=0 # for bottom layer
    if not gomofs_coordinaterange(lat,lon):
        print('lat and lon out of range in gomofs')
        return np.nan
    if date_time<datetime.datetime.strptime('2018-07-01 00:00:00','%Y-%m-%d %H:%M:%S'):
        print('Time out of range, time start :2018-07-01 00:00:00z')
        return np.nan
    if date_time>datetime.datetime.utcnow()+datetime.timedelta(days=3): #forecast time under 3 days
        print('beyond the forecast time of 3 days')
        return np.nan
    if date_time>datetime.datetime.utcnow():
        url=get_gomofs_url_forecast(datetime.datetime.utcnow(),date_time)
    else:
        url=get_gomofs_url(date_time)
    #start download data
    nc=netCDF4.Dataset(str(url))
    gomofs_lons=nc.variables['lon_rho'][:]
    gomofs_lats=nc.variables['lat_rho'][:]
    gomofs_temp=nc.variables['temp']
    #caculate the index of the nearest four points using a "find_nd" function in Lei Zhao's conversion module   
    target_distance=2*dist(lat1=gomofs_lats[0][0],lon1=gomofs_lons[0][0],lat2=gomofs_lats[0][1],lon2=gomofs_lons[0][1])
    eta_rho,xi_rho=find_nd(target=target_distance,lat=lat,lon=lon,lats=gomofs_lats,lons=gomofs_lons)
    if dist(lat1=lat,lon1=lon,lat2=gomofs_lats[eta_rho][xi_rho],lon2=gomofs_lons[eta_rho][xi_rho])>mindistance:
        print('THE location is out of range')
        return np.nan
    temperature=gomofs_temp[0][rho_index][eta_rho][xi_rho]
    return temperature


def gomofs_coordinaterange(lat,lon):
    f1=-0.7490553378867058*lat-lon-40.98355685763821<=0
    f2=-0.5967392371008197*lat-lon-36.300860518805024>=0
    f3=2.695505391925802*lat-lon-188.76889647321198<=0
    f4=2.689125428655328*lat-lon-173.5017523298927>=0
    if f1 and f2 and f3 and f4:
        return True
    else:
        return False

In [0]:
#@title doppio { form-width: "5%" }
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 21 09:10:47 2018
@author: Lei Zhao with help from JiM and Vitalii

Modifications by Lei Zhao Feb 27 2019
-updates the method of calculate the layer_index in function of get doppio
-updated the method of calculate the min,second small and third small distance and index

Modifications by lei Zhao Mar 20, 2019
-updated the way to get temperature quicker

Modifications by JiM Mar 21, 2019
-just added some more documentation and spelling changes
"""
#!pip install netCDF4
import netCDF4
import datetime
#import zlconversions as zl  # this is a set of Lei Zhao's functions that must be in same folder 
import numpy as np
def get_doppio_url(date):
    url='http://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/his/runs/History_RUN_2018-11-12T00:00:00Z'
    return url.replace('2018-11-12',date)
def get_doppio(time='2018-11-12 12:00:00',lat=0,lon=0,depth='bottom',fortype='temperature'):
    """
    notice:
        the format of time is like "%Y-%m-%d %H:%M:%S" this time is utctime  or it can also be datetime
        the depth is under the bottom depth
    the module only output the temperature of point location
    if fortype ='temperature',only return temperature, else return temperature and depth
    """
    #if depth==99999:
    #   depth='bottom'
    if not doppio_coordinate(lat,lon):
        print('the lat and lon out of range in doppio')
        return np.nan,np.nan
    if type(time)==str:
        date_time=datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S') # transform time format
    elif type(time)==datetime.datetime:
        date_time=time
    else:
        print('check the type of input time in get_doppio')
    for m in range(0,7):
        try:
            url_time=(date_time-datetime.timedelta(days=m)).strftime('%Y-%m-%d')#
            url=get_doppio_url(url_time)
            #get the data 
            nc=netCDF4.Dataset(url)
            lons=nc.variables['lon_rho'][:]
            lats=nc.variables['lat_rho'][:]
            doppio_time=nc.variables['time']
            doppio_rho=nc.variables['s_rho']
            doppio_temp=nc.variables['temp']
            doppio_h=nc.variables['h']
        except:
            continue
        min_diff_time=abs(datetime.datetime(2017,11,1,0,0,0)+datetime.timedelta(hours=int(doppio_time[0]))-date_time)
        min_diff_index=0
        for i in range(1,len(doppio_time)):
            diff_time=abs(datetime.datetime(2017,11,1,0,0,0)+datetime.timedelta(hours=int(doppio_time[i]))-date_time)
            if diff_time<min_diff_time:
                min_diff_time=diff_time
                min_diff_index=i
        #calculate the min,second small and third small distance and index
        target_distance=dist(lat1=lats[0][0],lon1=lons[0][0],lat2=lats[0][1],lon2=lons[0][1])
        index_1,index_2=find_nd(target=target_distance,lat=lat,lon=lon,lats=lats,lons=lons)

        #calculate the optimal layer index added this section Feb 2020
        doppio_depth=nc['h'][index_1][index_2]
        if depth > doppio_depth:# case of bottom
            S_coordinate=1
        else:
            S_coordinate=float(depth)/float(doppio_depth)
        if 0<=S_coordinate<1:
            layer_index=39-int(S_coordinate/0.025)#doppio_temp=temp[itime,39-int(S_coordinate/0.025),index_1,index_2]# because there are 0.025 between each later
        elif S_coordinate==1:
            layer_index=0#doppio_temp=temp[itime][0][index_1][index_2]
        else:
            layer_index=0#doppio_temp=temp[itime][0][index_1][index_2]
        #return doppio_temp
        #layer_index=0  #specify the initial layer index
        '''if depth!='bottom':
            h_distance=depth+doppio_rho[0]*doppio_h[index_1,index_2]  #specify the initial distanc of high
            for i in range(len(doppio_rho)):
                if abs(depth+doppio_rho[0]*doppio_h[index_1,index_2])<=h_distance:
                    h_distance=depth+doppio_rho[i]*doppio_h[index_1,index_2]
                    layer_index=i
                if depth>doppio_h[index_1,index_2]:
                    print ("the depth is out of the depth of bottom:"+str(doppio_h[index_1,index_2]))
        '''
        if index_1==0:
            index_1=1
        if index_1==len(lats)-1:
            index_1=len(lats)-2
        if index_2==0:
            index_2=1
        if index_2==len(lats[0])-1:
            index_2=len(lats[0])-2
        while True:
            point=[[lats[index_1][index_2],lons[index_1][index_2],doppio_temp[min_diff_index,layer_index,index_1,index_2]],\
            [lats[index_1-1][index_2],lons[index_1-1][index_2],doppio_temp[min_diff_index,layer_index,(index_1-1),index_2]],\
            [lats[index_1+1][index_2],lons[index_1+1][index_2],doppio_temp[min_diff_index,layer_index,(index_1+1),index_2]],\
            [lats[index_1][index_2-1],lons[index_1][index_2-1],doppio_temp[min_diff_index,layer_index,index_1,(index_2-1)]],\
            [lats[index_1][index_2+1],lons[index_1][index_2+1],doppio_temp[min_diff_index,layer_index,index_1,(index_2+1)]]]
            break
        point_temp=fitting(point,lat,lon)
        if np.isnan(point_temp):
            continue
        if min_diff_time<datetime.timedelta(hours=1):
            break
    if fortype=='temperature':
        return point_temp
    else:
        return point_temp,doppio_h[index_1,index_2]

def fitting(point,lat,lon):
#represent the value of matrix
    ISum = 0.0
    X1Sum = 0.0
    X2Sum = 0.0
    X1_2Sum = 0.0
    X1X2Sum = 0.0
    X2_2Sum = 0.0
    YSum = 0.0
    X1YSum = 0.0
    X2YSum = 0.0

    for i in range(0,len(point)):
        
        x1i=point[i][0]
        x2i=point[i][1]
        yi=point[i][2]

        ISum = ISum+1
        X1Sum = X1Sum+x1i
        X2Sum = X2Sum+x2i
        X1_2Sum = X1_2Sum+x1i**2
        X1X2Sum = X1X2Sum+x1i*x2i
        X2_2Sum = X2_2Sum+x2i**2
        YSum = YSum+yi
        X1YSum = X1YSum+x1i*yi
        X2YSum = X2YSum+x2i*yi

#  matrix operations
# _mat1 is the mat1 inverse matrix
    m1=[[ISum,X1Sum,X2Sum],[X1Sum,X1_2Sum,X1X2Sum],[X2Sum,X1X2Sum,X2_2Sum]]
    mat1 = np.matrix(m1)
    m2=[[YSum],[X1YSum],[X2YSum]]
    mat2 = np.matrix(m2)
    _mat1 =mat1.getI()
    mat3 = _mat1*mat2

# use list to get the matrix data
    m3=mat3.tolist()
    a0 = m3[0][0]
    a1 = m3[1][0]
    a2 = m3[2][0]
    y = a0+a1*lat+a2*lon

    return y

def doppio_coordinate(lat,lon):
    f1=-0.8777722604596849*lat-lon-23.507489034447012>=0
    f2=-1.072648270137022*lat-40.60872567829448-lon<=0
    f3=1.752828434063416*lat-131.70051451008493-lon>=0
    f4=1.6986954871237598*lat-lon-144.67649951783605<=0
    if f1 and f2 and f3 and f4:
        return True
    else:
        return False


In [0]:
#@title fvcom

# routines to extract FVCOM
# written by several interns 
# cleaned up by JiM in May 2020

from datetime import datetime as dt
from datetime import timedelta as td
import numpy as np
import netCDF4

def nearlonlat(lon,lat,lonp,latp): # needed for the next function get_FVCOM_bottom_temp
    """
    i=nearlonlat(lon,lat,lonp,latp) change
    find the closest node in the array (lon,lat) to a point (lonp,latp)
    input:
        lon,lat - np.arrays of the grid nodes, spherical coordinates, degrees
        lonp,latp - point on a sphere
        output:
            i - index of the closest node
            For coordinates on a plane use function nearxy           
            Vitalii Sheremet, FATE Project  
    """
    cp=np.cos(latp*np.pi/180.)
    # approximation for small distance
    dx=(lon-lonp)*cp
    dy=lat-latp
    dist2=dx*dx+dy*dy
    i=np.argmin(dist2)
    return i

def get_FVCOM_url(dtime):
    # "dtime" is in the form of datetime
    # get fvcom url based on time wanted
    if (dtime-dt.now())>td(days=-2): # forecast field
        url='http://www.smast.umassd.edu:8080/thredds/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_GOM3_FORECAST.nc' 
    elif (dtime>=dt(2016,7,1)) and (dtime<dt(2020,3,1)):
        url='http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/NECOFS/Archive/NECOFS_GOM/2019/gom4_201907.nc'
        url=url.replace('201907',dtime.strftime('%Y%m'))
        url=url.replace('2019',dtime.strftime('%Y'))
    elif dtime<=dt(2016,1,1): # 30 year hindcast
        url = 'http://www.smast.umassd.edu:8080/thredds/dodsC/fvcom/hindcasts/30yr_gom3'
    else:
        url=np.nan # not available
    return url

def get_FVCOM_temp(dtime,lati,loni,depth): # gets modeled temp using GOM3 forecast
        '''
        Taken primarily from Rich's blog at: http://rsignell-usgs.github.io/blog/blog/2014/01/08/fvcom/ on July 30, 2018
        where lati and loni are the position of interest, dtime is the datetime, and depth is "99999" for bottom
        '''
        urlfvcom=get_FVCOM_url(dtime)
        nc = netCDF4.Dataset(urlfvcom).variables
        #first find the index of the grid 
        lat = nc['lat'][:]
        lon = nc['lon'][:]
        inode = nearlonlat(lon,lat,loni,lati)
        #second find the index of time
        time_var = nc['time']
        itime = netCDF4.date2index(dtime,time_var,select='nearest')# where startime in datetime
        # figure out layer from depth
        w_depth=nc['h'][inode]
        if depth==99999: # for bottom
            layer=-1
        else:
            layer=int(round(depth/w_depth*45.))
        return nc['temp'][itime,layer,inode].data.flatten()[0] # had to add this suffix to extract value from a masked array


In [0]:
#@title get_models_cell { form-width: "10%" }
#!pip install netCDF4
'''
import sys
sys.path.append('/content/drive/My Drive/colab/')
import gomofs_modules
import doppio_modules
import fvcom_modules
'''
from datetime import datetime as dt

#HARDCODES ############
# Gets imput from terminal
lat=41.9
lon=-70.25
depth=99999#99999 for bottom
datet=dt(2019,8,15,0,0,0) #GMT time
clim='no' # set to yes if you have climatology files handy
clim_files_directory='/net/data5/jmanning/clim/'
#####################


# get DOPPIO
try:
  tdo=get_doppio(datet,lat,lon,depth,'temperature')
  print('doppio = ','%.3f' % tdo)
except:
  print('doppio not available?')

# get FVCOM
try:
    #url=fvcom_modules.get_FVCOM_url(datet)
    tf=get_FVCOM_temp(datet,lat,lon,depth)# 
    print('fvcom =','%.3f' % tf)
except:
    print('fvcom not available?')

# get GOMOFS bottom temp
try:
    tg=get_gomofs(datet,lat,lon,mindistance=20)
    print('gomofs = ','%.3f' % tg)
except:
    print('gomofs not available?')

doppio =  8.771
fvcom = 11.385
